My very first project with supervised deep learning. I have been reading "Deep Learning with Python" by françois chollet. I am not finished with it, but I really wanted to create my own solution on a given data set. 

The dataset I will be using is https://www.kaggle.com/sonalidasgupta95/churn-prediction-of-bank-customers/downloads/churn-prediction-of-bank-customers.zip/1
It will be a binary classification problem to determine if a customer will exit the bank.

In [ ]:
import pandas as pd

data = pd.read_csv('Churn_Modelling.csv')

data.head()

Just looking at the data, I will parseout columns 0, 1, 2: Row Number, CustomerId, Surname as these features should have no impact on the prediction of someone exiting the bank. Thus, I am going to use CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, and EstimatedSalary to determine whether the customer will exit. Also, I will preprocess the data that are inconvienient for the network
    - Geography: France will be 0, Spain will be 1, and Germany will be 2
    - Gender: Female will be 0, Male will be 1
    - Balance: Get the mean then divide by the standard deviation
    - EstimatedSalary: Get the mean then divide by the standard deviation

In [2]:
import csv
import numpy as np

# Getting Data
with open('Churn_Modelling.csv', 'r') as f:
    data = list(csv.reader(f, delimiter=';'))

for i in range(len(data)):
    data[i] = data[i][0].split(',')
    
    # Switching gender to numeric
    if data[i][5] == "Female":
        data[i][5] = 0
    elif data[i][5] == "Male":
        data[i][5] = 1
    
    # Switching geography to numeric
    if data[i][4] == "France":
        data[i][4] = 0
    elif data[i][4] == "Spain":
        data[i][4] = 1
    elif data[i][4] == "Germany":
        data[i][4] = 2
    
    data[i].pop(2)
    data[i].pop(1)
    data[i].pop(0)
    
    if i != 0:
        for j in range(len(data[i])):
            data[i][j] = float(data[i][j])
            
print(data[0])
data.pop(0)
data = np.array(data, np.float64)
print(data[0])

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']
[6.1900000e+02 0.0000000e+00 0.0000000e+00 4.2000000e+01 2.0000000e+00
 0.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0134888e+05
 1.0000000e+00]


Above shows you an example of data formatted in which order. With the data dissected, I am going to cut the data into a training data and testing data. 75% of the data will be used for training and the rest for testing. I will also create a target array for training and testing.

In [3]:
import math

# Splitting the data into train and test
np.random.shuffle(data)

# 75% of the data will be training data
numTrain = math.floor(len(data) * .75)

trainData = data[:numTrain]
testData = data[numTrain:]

trainTarget = []
testTarget = []

for i in range(len(trainData)):
    trainTarget.append(trainData[i][10])
for i in range(len(testData)):
    testTarget.append(testData[i][10])
    
trainData = np.delete(trainData, 10, 1)
testData = np.delete(testData, 10, 1)

Having split up the data, it is ready for it's final data preprocessig. Data features: CreditScore, Balance, and Salary are normalized. By subtracting the feature's mean off it's own and dividing it by the standard deviation, I can make the range of the data smaller. This is so that the network does not encounter wide extreme numbers on input.

In [ ]:
# Normalization
credit = []
salary = []
balance = []

for i in range(len(trainData)):
    credit.append(trainData[i][0])
    balance.append(trainData[i][5])
    salary.append(trainData[i][9])

creditMean = np.mean(credit)
salaryMean = np.mean(salary)
balanceMean = np.mean(balance)
creditStd = np.std(credit)
salaryStd = np.std(salary)
balanceStd = np.std(balance)

trainData[:, 0] -= creditMean
trainData[:, 0] /= creditStd
trainData[:, 5] -= balanceMean
trainData[:, 5] /= balanceStd
trainData[:, 9] -= salaryMean
trainData[:, 9] /= salaryStd

print(trainData[:5, 0])
print(trainData[:5, 5])
print(trainData[:5, 9])

Above shows how the range of the features have drastically decreased. Credit score, salary, and balance are now all normalized and ready to be inputted into our model. 
The network model has 4 layers:
    Input Layer : size of 10
    Hidden Layer 1 : size of 64
    Hidden LAyer 2 : size of 32
    Output LAyer : size of 1
    
As for the compilers, I am testing them.

In [5]:
import keras
from keras import models
from keras import layers
from keras import optimizers

keras.backend.clear_session()

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

epochs = 20
model_hist = model.fit(trainData, trainTarget, epochs=20, batch_size=128, validation_data=(testData, testTarget))

W0728 20:23:53.516337  6680 deprecation_wrapper.py:119] From C:\Users\theda\Anaconda3\envs\hello-tf\lib\site-packages\keras\backend\tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0728 20:23:53.517882  6680 deprecation_wrapper.py:119] From C:\Users\theda\Anaconda3\envs\hello-tf\lib\site-packages\keras\backend\tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0728 20:23:53.565684  6680 deprecation_wrapper.py:119] From C:\Users\theda\Anaconda3\envs\hello-tf\lib\site-packages\keras\backend\tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 20:23:53.568830  6680 deprecation_wrapper.py:119] From C:\Users\theda\Anaconda3\envs\hello-tf\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.comp

Train on 7500 samples, validate on 2500 samples
Epoch 1/20
7500/7500 [==============================] - 0s 66us/step - loss: 12.7135 - acc: 0.2025 - val_loss: 12.6391 - val_acc: 0.2072
Epoch 2/20
7500/7500 [==============================] - 0s 16us/step - loss: 12.7135 - acc: 0.2025 - val_loss: 12.6391 - val_acc: 0.2072
Epoch 3/20
7500/7500 [==============================] - 0s 14us/step - loss: 12.7135 - acc: 0.2025 - val_loss: 12.6391 - val_acc: 0.2072
Epoch 4/20
7500/7500 [==============================] - 0s 13us/step - loss: 12.7135 - acc: 0.2025 - val_loss: 12.6391 - val_acc: 0.2072
Epoch 5/20
7500/7500 [==============================] - 0s 11us/step - loss: 12.7135 - acc: 0.2025 - val_loss: 12.6391 - val_acc: 0.2072
Epoch 6/20
7500/7500 [==============================] - 0s 14us/step - loss: 12.7135 - acc: 0.2025 - val_loss: 12.6391 - val_acc: 0.2072
Epoch 7/20
7500/7500 [==============================] - 0s 12us/step - loss: 12.7135 - acc: 0.2025 - val_loss: 12.6391 - val_acc: 

In [ ]:
import matplotlib.pyplot as plt

model_acc = model_hist.history['acc']
model_loss = model_hist.history['loss']
model_val_acc = model_hist.history['val_acc']
model_val_loss = model_hist.history['val_loss']

epochs = range(1, len(model_acc) + 1)
# b+ is for "blue cross"
plt.plot(epochs, model_val_loss, 'b+', label='Val Loss')
plt.plot(epochs, model_loss, 'b', label='Loss')
plt.xlabel('Epochs')
plt.ylabel('Validation loss')
plt.legend()

plt.show()

trainTarget.count(1)/len(trainTarget)